<h1> Soal 1: Pemahaman Tentang Model Evaluasi</h1>

Jawab pertanyaan di bawah ini dengan bahasa masing-masing?

1. Apa perbedaan antara data latih, data validasi, dan data test?
2. Bagaimana cara kita menilai performa suatu model?
3. Apa itu Confusion Matrix? Jelaskan secara lengkap!
4. Apa itu Classification Report dari sklearn?

Jawab:
- Data training/latih adalah data yang digunakan untuk training model, data validasi digunakan untuk proses validasi model dan mencegah overfitting dan terakhir adalah data testing yang digunakan untuk testing model, sebagai simulasi penggunaan model pada dunia nyata. Data testing tidak boleh pernah dilihat oleh model sebelumnya.
- Cara untuk menilai performa suatu model bisa menggunakan Metrics yaitu fungsi untuk menghitung performa suatu model.
- Confusion matrix digunakan untuk mengevaluasi performa model terhadap data tes yang telah diketahui nilai sebenarnya dimana terdapat 4 istilah sebagai representasi hasil proses klasifikasi pada confusion matrix. Keempat istilah tersebut adalah True Positive (TP), True Negative (TN), False Positive (FP) dan False Negative (FN). Untuk mengingat penilaian yang lebih mudah bisa disebut jika diawali dengan True maka prediksinya adalah benar, entah diprediksi terjadi atau tidak terjadi, jika diawali dengan False maka prediksinya adalah salah dan positif dan negatif merupakan hasil prediksi dari model.

---

<h1>Soal 2: Aplikasi Model Evaluasi</h1>

Kali ini kita akan menggunakan data untuk memprediksi kelangsungan hidup pasien yang telah mengalami operasi payudara. Dengan informasi yang dimiliki terkait pasien, kita akan membuat model untuk memprediksi apakah pasien akan bertahan hidup dalam waktu lebih dari 5 tahun atau tidak.

Lebih Lengkapnya kalian bisa membaca informasi tentang dataset di link berikut: https://raw.githubusercontent.com/jbrownlee/Datasets/master/haberman.names

Buat model Klasifikasi (Model/Algoritma Bebas) untuk memprediksi status pasien dengan ketentuan sebagai berikut:

1. Bagi kedua data ini menjadi data training dan data test dengan test_size=0.25.
3. Pelajar tentang metrics roc_auc_score kemudian buatlah model dan evaluasi dengan menggunakan teknik cross-validation dengan scoring 'roc_auc'. Baca https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html untuk menggunakan metric roc_auc saat cross-validation.
3. Berapa score rata2 dari model dengan teknik cross-validation tersebut?
4. Prediksi data test dengan model yang telah kalian buat!
5. Bagaimana hasil confusion matrix dari hasil prediksi tersebut?
6. Bagaimana classification report dari hasil prediksi tersebut?
5. Seberapa baik model anda dalam memprediksi seorang pasien mempunyai status positive?
6. Seberapa baik model anda dalam memprediksi seorang pasien mempunyai status negatif?

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import roc_curve,auc,confusion_matrix,classification_report
from sklearn.model_selection import StratifiedKFold

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/haberman.csv'
list_cols = ['Age', "Patient's Years", "N_positive_ax", "survival_status"]
df = pd.read_csv(url, names=list_cols)

In [64]:
df['survival_status'].value_counts()

1    225
2     81
Name: survival_status, dtype: int64

In [65]:
# Code here
x = df.drop('survival_status', axis=1)
y = df['survival_status']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,stratify=y)
clf = KNeighborsClassifier(n_neighbors=5)

clf = clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)

print('Accuracy Model: ',metrics.accuracy_score(y_test,y_pred))

Accuracy Model:  0.7012987012987013


In [97]:
from sklearn.model_selection import cross_val_score

clf = KNeighborsClassifier(n_neighbors=5)

cv_res = cross_val_score(clf, x, y, cv=5, scoring='roc_auc')

print('Cross-Validation Mean : ',cv_res.mean())

Cross-Validation Mean :  0.6473039215686275


In [67]:
params_grid = {
    'n_neighbors':np.arange(2,50),'weights':['uniform','distance']
}
gscv = GridSearchCV(clf, params_grid, cv=10, scoring='f1')
gscv.fit(x_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [77]:
y_pred = gscv.predict(x_test)

confusion_matrix(y_test, y_pred)

array([[50,  7],
       [16,  4]], dtype=int64)

In [75]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.76      0.88      0.81        57
           2       0.36      0.20      0.26        20

    accuracy                           0.70        77
   macro avg       0.56      0.54      0.54        77
weighted avg       0.66      0.70      0.67        77



In [88]:
TP = 4
FN = 16
FP = 7
TF = 50

#precision untuk prediksi dengan status negatif
precision = TF/(TF+FN)
print('Prediksi model untuk status negatif',precision)

#precision untuk prediksi dengan status positif
precision = TP/(TP+FP)
print('Prediksi model untuk status positif :',precision)

Prediksi model untuk status negatif 0.7575757575757576
Prediksi model untuk status positif : 0.36363636363636365


---

<h1> Soal 3: Pemahaman Tentang Model Selection</h1>

Jelaskan dengan bahasa sendiri!

1. Apa itu Bias dan Variance?
2. Apa itu Overfitting dan Underfitting?
3. Apa yang bisa kita lakukan untuk mengatur kompleksitas dari model?
4. Bagaimana model yang baik?
5. Kapan kita menggunakan GridSearchcv dan kapan menggunakan RandomizedSearchCV?

Jawab:
- Bias adalah asumsi penyederhanaan yang dibuat oleh model untuk membuat fungsi target lebih mudah untuk diperkirakan. Varians adalah jumlah estimasi fungsi target yang akan berubah berdasarkan data pelatihan yang berbeda.
- Overfitting mengacu pada model yang memodelkan data training dengan terlalu baik. Overfitting terjadi saat model mempelajari detail dan noise dalam data training sejauh hal itu berdampak negatif pada performa model pada data baru. Underfitting mengacu pada model yang tidak dapat memodelkan data training atau menggeneralisasi ke data baru. Model pembelajaran mesin yang kurang cocok bukanlah model yang sesuai dan akan terlihat jelas karena akan memiliki performa yang buruk pada data pelatihan.
- pemilihan model dan subset linier, metode penyusutan (termasuk regularisasi), dan pengurangan dimensi.
- Dalam klasifikasi mungkin cara untuk mengetahui model yang baik bisa di lihat dari score akurasinya, jika 100% berarti baik dan masalah regresi mungkin score 0.0 error yang paling baik tetapi skor model yang baik terlihat relatif terhadap baseline pada dataset spesifik yang masing-masing miliki.
- Penggunaan GridSearchCV mencari parameter terbaik untuk pengklasifikasi yang ingin kita gunakan dan RandomizedSearchCV mencoba kombinasi acak di antara rentang nilai yang ditentukan untuk hyper-parameter.

---

<h1> Soal 4: Aplikasi Model Selection</h1>

1. Bagi kedua data berikut ini menjadi data training dan data test dengan test_size=0.25.
2. Gunakan algoritma KNN sebagai model classifier.
3. Gunakan fungsi GridSearchCV untuk hyperparameter tuning dan model selection.
4. jumlah fold bebas!, gunakan scoring 'roc_auc'
5. Definisikan kombinasi hyperparameter untuk model selection dengan GridSearchCV. kombinasi Hyperparameter bebas, baca lagi dokumentasi KNN di link berikut https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html untuk memahami lagi jenis2 hyperparameter di algorithma KNN.
6. Latih model terhadap data training.
7. Apa hyperparameter terbaik untuk kombinasi hyperparameter kalian?
8. Berapa score validasi terbaik dari model tersebut?
9. Prediksi probabilitasi output dari model yang telah di buat terhadap data test. note : gunakan method .predict_proba() untuk menghasilkan output probabilitas
10. Perhatikan bahwa hasil prediksi ada 2, dimana masing2 adalah nilai probabilitas untuk setiap class label. Ambil nilai probabilitas pasien phositive meninggal dalam waktu kurang dari 5 tahun. note : gunakan bantuan attirubte .classes_ untuk mengetahui urutan label dari hasil prediksi probabilitas.
11. Berapa nilai score roc_auc untuk data test?
12. Apakah model anda termasuk baik, overtting, atau underfitting?
13. Ulangi tahap di atas namun kali ini menggunakan algoritma DecisionTreeClassifier dan kalian bisa menggunakan RandomizedSearchCV apabila process training lama. pelajari algoritma DecisionTreeClassifier di linkberikut: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?highlight=decisiontreeclassifier#sklearn.tree.DecisionTreeClassifier
14. Bandingkan scorenya dengan Algoritma KNN, mana yang lebih baik?

Note : Data Science adalah experiment, sangat di dimungkinkan memerlukan beberapa kali percobaan untuk mendapatkan hasil yang terbaik! Happy Coding :)

In [89]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/haberman.csv'
list_cols = ['Age', "Patient's Years", "N_positive_ax", "survival_status"]
df = pd.read_csv(url, names=list_cols)

In [90]:
df.head()

,Age,Patient's Years,N_positive_ax,survival_status
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1


In [92]:
# Code here
x = df.drop('survival_status', axis=1)
y = df['survival_status']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,stratify=y)

clf = KNeighborsClassifier(n_neighbors=5)

params_grid = {
    'n_neighbors':np.arange(2,50),'weights':['uniform','distance']
}
gscv = GridSearchCV(clf, params_grid, cv=10, scoring='f1')
gscv.fit(x_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [94]:
y_pred = gscv.predict(x_test)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1,
       1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [95]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.78      0.91      0.84        57
           2       0.50      0.25      0.33        20

    accuracy                           0.74        77
   macro avg       0.64      0.58      0.59        77
weighted avg       0.70      0.74      0.71        77



In [105]:
cv_res = cross_val_score(clf, x, y, cv=15, scoring='roc_auc')
print('Hasil scoring KNN : ',cv_res.mean())

Hasil scoring KNN :  0.6277037037037038


In [112]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=20)

params_grid = {
    'random_state':np.arange(2,50)
}
gscv = GridSearchCV(clf, params_grid, cv=10, scoring='f1')
gscv.fit(x_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=20,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'random_state': array([ 2,  3,  4,  5,  6,  7,  8,  9,

In [113]:
y_pred = gscv.predict(x_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.75      0.79      0.77        57
           2       0.29      0.25      0.27        20

    accuracy                           0.65        77
   macro avg       0.52      0.52      0.52        77
weighted avg       0.63      0.65      0.64        77



In [114]:
cv_res = cross_val_score(clf, x, y, cv=15, scoring='roc_auc')
print('Hasil scoring DecisionTreeClassifier : ',cv_res.mean())

Hasil scoring DecisionTreeClassifier :  0.4555555555555556
